In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.0 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from transformers import create_optimizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score, roc_curve, auc, f1_score,confusion_matrix
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import roc_auc_score,precision_score, recall_score, f1_score,accuracy_score

In [ ]:
def Evaluation (Method,Comment,Actual, Predicted,Scores):
    ROC_AUC=roc_auc_score(Actual,Predicted)
    Precision=precision_score(Actual,Predicted)
    Accuracy=accuracy_score(Actual,Predicted)
    Recall=recall_score(Actual,Predicted)
    F1=f1_score(Actual,Predicted)
    print('Accuracy=',Accuracy)
    Scores[Comment] = {'Method':Method,'ROC_AUC':ROC_AUC,'Accuracy':Accuracy,
                       'Precision':Precision,'Recall':Recall,'F1':F1}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisStockProject/dataset/pre_process_news_days.csv', encoding = "ISO-8859-1")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Split Data

In [ ]:
# Split the data into train, validation, and test sets
train_data = df[(df['Date'] >= '2008-08-08') & (df['Date'] <= '2013-12-31')]
val_data = df[(df['Date'] >= '2014-01-02') & (df['Date'] <= '2014-12-31')]
test_data = df[(df['Date'] >= '2015-01-02') & (df['Date'] <= '2016-07-01')]

In [ ]:
X_train = train_data.news_stemmed.values
X_val = val_data.news_stemmed.values
X_test = test_data.news_stemmed.values

In [ ]:
#Input Layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

#Preprocessing Layer
preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)

#Encoder Layer
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/2", trainable=True, name='smallBERT_encoder')
outputs = encoder(encoder_inputs)

In [ ]:
sc={}
for i, lbl in enumerate(['Label','Label_1day','Label_2day', 'Label_3day', 'Label_4day', 'Label_5day']):

#Classification Layers
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.5)(net)
  net = tf.keras.layers.Dense(516, activation='relu')(net)
  net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
  model = tf.keras.Model(inputs=[text_input], outputs=[net])

  WARMUP_STEPS =int(0.2*len(X_train))
  batch_size = 32
  batches_per_epoch = len(X_train) // batch_size
  total_train_steps = int(batches_per_epoch * 4)

  optimizer, schedule = create_optimizer(
      init_lr=2e-5, num_warmup_steps=WARMUP_STEPS, num_train_steps=total_train_steps)

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='acc')
  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  Y_train = train_data[lbl]
  Y_val = val_data[lbl]
  Y_test = test_data[lbl]

  np.random.seed(80)
  history = model.fit(X_train, Y_train,
                        callbacks=[es],
                        validation_data=(X_val, Y_val),
                        epochs=4)
  predictions = model.predict(X_test)
  y_pred = np.round(predictions)
  y_pred = np.argmax(y_pred, axis=1)

  Evaluation ('Bert ',lbl,Y_test, y_pred,sc)

Epoch 1/4
43/43 [==============================] - 36s 537ms/step - loss: 0.6991 - acc: 0.5298 - val_loss: 0.7068 - val_acc: 0.4484
Epoch 2/4
43/43 [==============================] - 20s 470ms/step - loss: 0.7000 - acc: 0.5350 - val_loss: 0.7143 - val_acc: 0.4484
Epoch 3/4
43/43 [==============================] - 20s 475ms/step - loss: 0.7092 - acc: 0.4923 - val_loss: 0.6956 - val_acc: 0.4841
Epoch 4/4
12/12 [==============================] - 3s 197ms/step
Accuracy= 0.49206349206349204
Epoch 1/4
43/43 [==============================] - 36s 537ms/step - loss: 0.7253 - acc: 0.4952 - val_loss: 0.6894 - val_acc: 0.5357
Epoch 2/4
43/43 [==============================] - 21s 494ms/step - loss: 0.7043 - acc: 0.5202 - val_loss: 0.6965 - val_acc: 0.4762
Epoch 3/4
43/43 [==============================] - 21s 498ms/step - loss: 0.7014 - acc: 0.5305 - val_loss: 0.6882 - val_acc: 0.5516
Epoch 4/4
12/12 [==============================] - 3s 192ms/step
Accuracy= 0.5079365079365079
Epoch 1/4
43/43 [==

In [ ]:
df_sc = pd.DataFrame.from_dict(sc, orient='index')
df_sc.sort_values(by=['ROC_AUC','Accuracy'],ascending=False,inplace=True)
df_sc

,Method,ROC_AUC,Accuracy,Precision,Recall,F1
Label_2day,Bert,0.5,0.547619,0.0,0.0,0.0
Label_4day,Bert,0.5,0.534392,0.0,0.0,0.0
Label_3day,Bert,0.5,0.531746,0.0,0.0,0.0
Label_5day,Bert,0.5,0.531746,0.0,0.0,0.0
Label_1day,Bert,0.5,0.507937,0.0,0.0,0.0
Label,Bert,0.5,0.492063,0.0,0.0,0.0
